# LoRA

### LoRA를 활용한 GPT-2 감성 분석 모델 튜닝

In [1]:
!pip install peft datasets transformers hf_transfer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 77.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 98.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 17.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 82.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 110.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 81.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 125.7 MB/s  0:00:00m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.5/803.5 kB 37.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 105.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25/25 [datasets]/25 [datasets]e]s]ub]


In [2]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
from datasets import load_dataset

In [3]:
# 환경 변수 설정 (local에서 진행할 때는 token 설정 필요)
# from huggingface_hub import login
# login(token="hf_XXX") # 실제 HF_TOKEN 입력

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [5]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

### 1. 기반 모델 load

In [6]:
model_name = 'gpt2'

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
base_model.config.pad_token_id = tokenizer.pad_token_id

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 2. LoRA 설정 (LoraConfig)

In [7]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=['c_attn', 'c_fc', 'c_proj'],
    lora_dropout=0.1,
    task_type='SEQ_CLS',
    bias='none'
)

In [8]:
# 기반 모델 + LoRA 설정 == 학습할 모델
model = get_peft_model(base_model, lora_config).to(device)

/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/layer.py:2174: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


### 3. 데이터 준비

In [9]:
dataset = load_dataset('imdb')

README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [10]:
# 학습용 데이터 준비
# 샘플링 편향 방지
pos_sample = [data for data in dataset['train'] if data['label'] == 1][:500]
neg_sample = [data for data in dataset['train'] if data['label'] == 0][:500]

# X, y 데이터 구분
train_texts = [data['text'] for data in pos_sample + neg_sample]
train_labels = [data['label'] for data in pos_sample + neg_sample]

In [11]:
# 검증용 데이터 준비
# 샘플링 편향 방지
pos_eval = [data for data in dataset['test'] if data['label'] == 1][:500]
neg_eval = [data for data in dataset['test'] if data['label'] == 0][:500]

# X, y 데이터 구분
eval_texts = [data['text'] for data in pos_eval + neg_eval]
eval_labels = [data['label'] for data in pos_eval + neg_eval]

In [12]:
def preprocess_data(texts, labels):
    encodings = tokenizer(
        texts,
        truncation=True,
        padding='max_length',
        max_length=512,
        return_tensors='pt'
    )
    encodings['labels'] = torch.tensor(labels, dtype=torch.long)

    return encodings

In [13]:
train_encodings = preprocess_data(train_texts, train_labels)
eval_encodings = preprocess_data(eval_texts, eval_labels)

In [14]:
class IMDBDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings['input_ids'])

    def __getitem__(self, idx):
        return {
            key: val[idx] for key, val in self.encodings.items()
        }

In [15]:
train_dataset = IMDBDataset(train_encodings)
eval_dataset = IMDBDataset(eval_encodings)

In [16]:
# 데이터 묶음 처리(batch 처리하는 함수)
def collate_fn(batch):
    batch = {
        key: torch.stack([item[key] for item in batch]) for key in batch[0]
    }
    return batch

### 4. 학습 준비

In [17]:
# 학습 파라미터 설정(학습에 대한 설정)
training_args = TrainingArguments(
    output_dir='./results',                      # 결과 저장 경로
    per_device_train_batch_size=4,               # 학습 배치 크기
    per_device_eval_batch_size=4,                # 검증 배치 크기
    num_train_epochs=5,                          # 학습 반복 횟수
    save_steps=100,                              # 저장 주기
    save_total_limit=2,                          # 최대 2개의 모델 저장
    eval_strategy='epoch',                       # 검증 주기
    logging_dir='./logs',                        # 로그 저장 경로
    logging_steps=10,                            # 로그 출력 주기
    fp16=torch.cuda.is_available()               # GPU 있으면 fp16, 없으면 자동으로 False
)

print(f"🔧 학습 설정 완료!")
print(f"   - Device: {device}")
print(f"   - FP16: {training_args.fp16}")
print(f"   - Epochs: {training_args.num_train_epochs}")
print(f"   - Batch size: {training_args.per_device_train_batch_size}")

🔧 학습 설정 완료!
   - Device: cuda
   - FP16: True
   - Epochs: 5
   - Batch size: 4


In [18]:
# Trainer 설정 (학습할 모델 + 학습 파라미터(학습 설정) + 데이터셋 + 데이터 처리 함수 + 데이터 묶음 처리 함수)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=collate_fn
)

In [ ]:
# local 에서 학습 진행되는 것을 확인하고, runpod 에서 실제 학습 진행
trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.923200,1.142838
2,0.778900,0.743863


/usr/local/lib/python3.12/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
